In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List, Tuple, Dict

In [42]:
import dataset as ds
import oai_utils as ou

In [34]:
with open("scr.txt", 'r', encoding='utf8') as f:
    key = f.read()

In [36]:
from openai import OpenAI
client = OpenAI(api_key=str(key))

In [ ]:
# completion = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {
#             "role": "user",
#             "content": "Write a haiku about recursion in programming."
#         }
#     ]
# )

# print(completion.choices[0].message)

ChatCompletionMessage(content='Functions call themselves,  \nA loop within the code flows,  \nDepths of logic hum.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


Using information from:
- https://openai.com/index/introducing-structured-outputs-in-the-api/


In [ ]:
# boolean_response_format = {
#     "type": "json_schema",
#     "json_schema": {
#         "name": "bool_response",
#         "strict": True,
#         "schema": {
#             "type": "object",
#             "properties": {
#                 "r": {
#                     "type": "boolean"
#                 }
#             },
#             "required": ["response"],
#             "additionalProperties": False
#         }
#     }
# }

In [38]:
# want to write code to match this format...
# # Input: "THE mat sat on the cat" Label: False
# Input: "the house is cold" Label: True

example_pairs = [
    ("the cat sat on the mat", True),
    ("THE DOG RAN IN THE PARK", False),
    ("THE mat sat on the cat", False),
    ("the house is cold", True),
]

test_pairs = {
    True: "The man ran along the road",
    False: "the man ran along the road",
}

In [ ]:
ms = ds.construct_test_messages(example_pairs, test_pairs)

In [40]:
(ms[True])

[{'role': 'system',
  'content': "\nYou are a helpful assistant given a classification challenge.\nLearn the rule from the following True False examples, then output the correct classification for the test input.\nYou should return a single {'True', 'False'} value.\n"},
 {'role': 'user',
  'content': 'Example pairs to learn from:\nInput: "the cat sat on the mat" Label: True\nInput: "THE DOG RAN IN THE PARK" Label: False\nInput: "THE mat sat on the cat" Label: False\nInput: "the house is cold" Label: True\n\nTest pairs to evaluate:\nInput: "the man ran along the road" Label: '}]

In [53]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=ms[False],
    #response_format=boolean_response_format,
    max_tokens=10,
    seed=0,
    response_format=ou.BOOLEAN_RESPONSE_FORMAT
)

print(completion.choices[0].message)

ChatCompletionMessage(content='{"response":false}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [55]:
resp = completion.choices[0].message.content

In [61]:
resp

'{"response":false}'

In [62]:
import json

In [64]:
json.loads(resp)['response']

False

In [ ]:
resp.replace('false', 'False')

'{"response":false}'

In [60]:
eval(capitalise_bools(resp))['response']

False

In [48]:
ou.extract_bool_response(completion)

False

In [67]:
ou.extract_bool_from_completion_json_format(completion)

False